Download main libraries and data

In [1]:
#install packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold


#!pip install --user nltk
!pip install yargy
#!pip install --user pymorphy2
#!pip install --user pymystem3
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, ToktokTokenizer
from yargy.tokenizer import MorphTokenizer
import re
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem

#import nltk
nltk.download("stopwords")
nltk.download('wordnet')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 45.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=318e17f51d5316736ae471e436c21eb083c705c62003276dafb95377ea7893a4
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
#google drive
FOLDER = "данные для проекта" 

PATH = "drive/MyDrive/" + FOLDER  + "/"
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append(PATH)

Mounted at /content/drive


In [3]:
df = pd.read_csv (PATH + "Real Estate Data V21.csv")

In [4]:
df #we need price_per_SQRT, Description, Property title and total_area

,Name,Property Title,Price,Location,Total_Area,Price_per_SQFT,Description,Baths,Balcony
0,Casagrand ECR 14,"4 BHK Flat for sale in Kanathur Reddikuppam, C...",₹1.99 Cr,"Kanathur Reddikuppam, Chennai",2583,7700.0,Best 4 BHK Apartment for modern-day lifestyle ...,4,Yes
1,"Ramanathan Nagar, Pozhichalur,Chennai",10 BHK Independent House for sale in Pozhichal...,₹2.25 Cr,"Ramanathan Nagar, Pozhichalur,Chennai",7000,3210.0,Looking for a 10 BHK Independent House for sal...,6,Yes
2,DAC Prapthi,"3 BHK Flat for sale in West Tambaram, Chennai",₹1.0 Cr,"Kasthuribai Nagar, West Tambaram,Chennai",1320,7580.0,"Property for sale in Tambaram, Chennai. This 3...",3,No
3,"Naveenilaya,Chepauk, Triplicane,Chennai",7 BHK Independent House for sale in Triplicane...,₹3.33 Cr,"Naveenilaya,Chepauk, Triplicane,Chennai",4250,7840.0,Entire Building for sale with 7 units of singl...,5,Yes
4,VGN Spring Field Phase 1,"2 BHK Flat for sale in Avadi, Chennai",₹48.0 L,"Avadi, Chennai",960,5000.0,"Property for sale in Avadi, Chennai. This 2 BH...",3,Yes
...,...,...,...,...,...,...,...,...,...
14523,"Krishna Park Extension, Tilak Nagar,New Delhi","2 BHK Flat for sale in Tilak Nagar, New Delhi",₹40.0 L,"Krishna Park Extension, Tilak Nagar,New Delhi",1053,3800.0,Looking for a good 2 BHK Apartment in Tilak Na...,3,Yes
14524,"Rawta, Jaffarpur Kalan,New Delhi",1 BHK Independent House for sale in Jaffarpur ...,₹14.0 L,"Rawta, Jaffarpur Kalan,New Delhi",472,2970.0,1 BHK Independent House for sale in Jaffarpur ...,2,Yes
14525,"Rani Garden, Geeta Colony,New Delhi","1 BHK Flat for sale in Geeta Colony, New Delhi",₹30.0 L,"Rani Garden, Geeta Colony,New Delhi",378,7940.0,"Property for sale in Geeta Colony, Delhi. This...",1,Yes
14526,"Lig flat rohini,Sector 16E, Sector 16 Rohini,N...","3 BHK Flat for sale in Sector 16 Rohini, New D...",₹60.0 L,"Lig flat rohini,Sector 16E, Sector 16 Rohini,N...",700,8570.0,Check out this 3 BHK Apartment for sale in Roh...,2,Yes


In [6]:
df.isna().sum()

Name              0
Property Title    0
Price             0
Location          0
Total_Area        0
Price_per_SQFT    0
Description       0
Baths             0
Balcony           0
dtype: int64

In [7]:
df = df[["Property Title", "Total_Area", "Price_per_SQFT", "Description"]]
df1 = df.head(n = 1000)

Text Preprocessing

In [8]:
#preprocess data
res = []
tknzr = MorphTokenizer()
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")
morph = Mystem()
for col_name, data in df["Description"].items():
  punct = '[!"#$%&()*\+,-\./:;<=>?@\[\]^_`{|}~„“«»†*\—/\-‘’]' #getting rid of punctuation
  clean_text = re.sub(punct, r' ', data)
  clean_text = re.sub('[0-9]+', '', clean_text) #getting rid of numbers

  clean_text = [_.value for _ in tknzr(clean_text)] #tokenization

  clean_text = [word for word in clean_text if len(word) > 2 if not word in stopwords.words('english')] #deleting stop words

  #clean_text = [stemmer.stem(word) for word in clean_text] #stemming

  clean_text = [lemmatizer.lemmatize(word) for word in clean_text] #lemmatization
  
  clean_text = [word.lower() for word in clean_text if word != ''] #convert to lower cases

  clean_text = ' '.join(clean_text) #joining tokens back to a string

  res.append(clean_text)

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [9]:
df["cleanedtext"] = res
df

<ipython-input-9-429aab7bdabe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cleanedtext"] = res


,Property Title,Total_Area,Price_per_SQFT,Description,cleanedtext
0,"4 BHK Flat for sale in Kanathur Reddikuppam, C...",2583,7700.0,Best 4 BHK Apartment for modern-day lifestyle ...,best bhk apartment modern day lifestyle availa...
1,10 BHK Independent House for sale in Pozhichal...,7000,3210.0,Looking for a 10 BHK Independent House for sal...,looking bhk independent house sale chennai you...
2,"3 BHK Flat for sale in West Tambaram, Chennai",1320,7580.0,"Property for sale in Tambaram, Chennai. This 3...",property sale tambaram chennai this bhk apartm...
3,7 BHK Independent House for sale in Triplicane...,4250,7840.0,Entire Building for sale with 7 units of singl...,entire building sale unit single bedroom unit ...
4,"2 BHK Flat for sale in Avadi, Chennai",960,5000.0,"Property for sale in Avadi, Chennai. This 2 BH...",property sale avadi chennai this bhk apartment...
...,...,...,...,...,...
14523,"2 BHK Flat for sale in Tilak Nagar, New Delhi",1053,3800.0,Looking for a good 2 BHK Apartment in Tilak Na...,looking good bhk apartment tilak nagar delhi t...
14524,1 BHK Independent House for sale in Jaffarpur ...,472,2970.0,1 BHK Independent House for sale in Jaffarpur ...,bhk independent house sale jaffarpur kalan del...
14525,"1 BHK Flat for sale in Geeta Colony, New Delhi",378,7940.0,"Property for sale in Geeta Colony, Delhi. This...",property sale geeta colony delhi this bhk apar...
14526,"3 BHK Flat for sale in Sector 16 Rohini, New D...",700,8570.0,Check out this 3 BHK Apartment for sale in Roh...,check bhk apartment sale rohini delhi this pro...


In [ ]:
ww = stopwords.words('english')
ww

In [ ]:
#если буквы русские
#morph = MorphAnalyzer()
#clean_text = [morph.parse(word)[0].normal_form for word in clean_text]

In [ ]:
#Mount the Drive to the Colab
drive.mount('/gdrive')

#Write the DataFrame to csv file
with open('/gdrive/My Drive/данные для проекта/final_dataset.csv', 'w') as f:
  df.to_csv(f)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


Build model

Plan:\
1)Apply Bag-of-Words\
2)Fit models with lasso-regularization and random forest regression\
3)Choose the most influencial features

In [ ]:
#1st model: trying to use bag-of-words. Here we are calculating feature importance with using only chi-square, no modeling.
vectorizer2 = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 5000)
X = vectorizer2.fit_transform(df['cleanedtext']) 
chi2_scores, p_values = chi2(X, df['Price_per_SQFT'].astype('int'))
significant_words = np.array(vectorizer2.get_feature_names_out())[chi2_scores.argsort()[::-1]][:50]
significant_words

array(['pillagana', 'besant', 'prabhadevi', 'jalladianpet', 'worli',
       'kilometer', 'sion', 'gujranwala', 'parel', 'bandra', 'bhuleshwar',
       'vasanth', 'inder', 'kamathipura', 'dadar', 'mauza', 'shibrampur',
       'sea', 'gamdevi', 'malabar', 'wadala', 'kamruddin', 'powai',
       'paranur', 'kanaka', 'chickpete', 'lines', 'colaba', 'sivanchetti',
       'doddakammanahalli', 'borivali', 'sewri', 'kanjurmarg', 'gorai',
       'tellapur', 'kalbalu', 'gulabi', 'adyar', 'mavalli', 'paikpara',
       'girgaon', 'nandambakkam', 'ghatkopar', 'umerkhadi', 'chembur',
       'tri', 'azadpur', 'kalbadevi', 'mahim', 'shaikpet'], dtype=object)

In [10]:
#1st model: trying to use bag-of-words. Here we are calculating whether the word is in the text or not and build a sparse matrix. Then build a dataframe from this sparse matrix.
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 8000) #, ngram_range=(2, 2)
train_data_features = vectorizer.fit_transform(res)

vectors = train_data_features.toarray()
names = vectorizer.get_feature_names_out()

bag = pd.DataFrame(vectors)
bag.columns = names

In [ ]:
bag

In [11]:
#create X and Y, then build a model
X = bag.copy()
Y = df['Price_per_SQFT'].astype('int').values.reshape(-1,1)

x_train, x_test, y_train, y_test = train_test_split (X, Y, test_size = 0.3)

In [22]:
#build lasso-regularization to find the most significant features
las = Lasso()
mod = las.fit(x_train, y_train)
las.score(x_train, y_train)
lasso_results = pd.Series(las.coef_, index = x_train.columns).sort_values()

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.360e+12, tolerance: 2.484e+09
  model = cd_fast.enet_coordinate_descent(


slr          -854662.538879
switching    -681584.406624
bharatmata   -621856.431215
chand        -571224.565910
goel         -484914.413915
                  ...      
kumar         598506.770120
mohanagar     668123.123968
ace           779673.547063
slab          808874.190582
survey        826305.814620
Length: 8000, dtype: float64

In [ ]:
importance = mod.coef_
imp = []
for i,v in enumerate(importance):
  print('Feature: %0d, Score: %.5f' % (i,v))
  imp.append('Feature: %0d, Score: %.5f' % (i,v))

In [30]:
lasso_res = pd.DataFrame({'col_name': mod.coef_}, index = x_train.columns).sort_values(by = 'col_name', ascending=False)
lasso_res

,col_name
survey,826305.814620
slab,808874.190582
ace,779673.547063
mohanagar,668123.123968
kumar,598506.770120
...,...
goel,-484914.413915
chand,-571224.565910
bharatmata,-621856.431215
switching,-681584.406624


In [ ]:
lasso_res[1:20]

In [ ]:
#Mount the Drive to the Colab
drive.mount('/gdrive')

#Write the DataFrame to csv file.
with open('/gdrive/My Drive/данные для проекта/lasso_results.csv', 'w') as f:
  lasso_results.to_csv(f)

In [26]:
reg = RandomForestRegressor (n_estimators = 100, bootstrap = True, max_depth = 5)
mod1 = reg.fit(x_train, y_train.ravel())
rf_results = pd.DataFrame({'col_name': mod1.feature_importances_}, index = x_train.columns).sort_values(by = 'col_name', ascending=False)

In [ ]:
rf_results[0:20]

In [18]:
#Mount the Drive to the Colab
drive.mount('/gdrive')

#Write the DataFrame to csv file.
with open('/gdrive/My Drive/данные для проекта/rf_results.csv', 'w') as f:
  rf_results.to_csv(f)

Mounted at /gdrive
